In [ ]:
import sys, os
import numpy as np
from pyemma.thermo import dtram
from pyemma import msm
import pyemma

In [ ]:
all_stride = [10,100,1000]
all_nstates = [10,30,50,100,200]

In [ ]:
for s in range(len(all_stride)):
        for n in range(len(all_nstates)):
                stride = all_stride[s]
                nstates = all_nstates[n]
                interval = 0.002*stride  # in ns (frames were saved every 2 ps)
                bin_min = 0.0     # first umbrella
                bin_max = 0.8     # last umbrella
                dx=(bin_max-bin_min)/nstates
                pmf_distances=np.arange(bin_min,bin_max+dx,dx)
                bincenters = (pmf_distances[0:-1]+pmf_distances[1:])/2.0
                # print pmf_distances
                # print bincenters

                assign = np.load('states_%d/frames_%d/clustered_matrix.npy'%(nstates,int(200000./stride)))  # assignment
                K = assign.shape[0] # termodynamic states
                if stride == 10000:
                        lagtimes = np.array([1,2,3,4,5,6,7,8,9,10])
                elif stride == 1000:
                        lagtimes = np.array([1,5,10,20,30,40,50,80,100])
                elif stride == 100:
                        lagtimes = np.array([1,15,50,100,200,300,400,500,800,1000])
                elif stride == 10:
                        lagtimes = np.array([1,20,50,100,200,500,800,1000,2000,3000,4000,5000,6000,7000,8000,10000])
                
                # let's load pre-computed bias
                bias = np.load('states_%d/frames_%d/bias.npy'%(nstates,int(200000./stride)))

                total_its = []
                bootstrap = False

                # this part is wrong and still needs to be fixed
                if bootstrap:
                        fold = 5
                        st = int(assign.shape[1]/fold)
                        for i in range(fold):
                                if i != fold-1:
                                        ass = np.concatenate((assign[:,:st*(i+1)],assign[:,st*(i+1)+st:]),axis=1)
                                else:
                                        ass = assign[:,st:]
                                ttrajs = []
                                dtrajs = []
                                for k in range(K):
                                        ttrajs.append(k*np.ones(ass.shape[1],dtype=int))
                                        dtrajs.append(ass[k])

                                motion = 4
                                dtram_obj = dtram(ttrajs, dtrajs, bias, lagtimes, unbiased_state=int(assign.shape[0])-1) # the unbiased ensemble is the last one

                                its=[[] for m in range(motion)]
                                for j in range(len(lagtimes)):
                                        time = dtram_obj[j].msm.timescales(motion)
                                        for l in range(motion):
                                                its[l].append(time[l])
                                total_its.append(its)

                        np.save('states_%d/frames_%d/total_its.npy'%(nstates,int(200000./stride)),total_its)

                # this part is finished
                else:
                        ttrajs = []    # create empty list of ttrajs
                        dtrajs = []    # create empty list of dtrajs
                        for k in range(K):   # K is the total number of thermodynamic states
                                ttrajs.append(k*np.ones(assign.shape[1],dtype=int))  # e.g. [[0,0,0,...],[1,1,1,1,...],...]
                                dtrajs.append(assign[k])     # the actual trajectory (clustered)
                        motion = 4    # 4 slow motions to compute
                        dtram_obj = dtram(ttrajs, dtrajs, bias, lagtimes, unbiased_state=int(assign.shape[0])-1) # the unbiased ensemble is the last one
                        its=[[] for m in range(motion)]    # create implited timescales list for slow motions
                        eigenvector = []   # create list of eigenvectors
                        for j in range(len(lagtimes)):
                                time = dtram_obj[j].msm.timescales(motion)   # compute implied timscales for a given lag time
                                ev = dtram_obj[j].msm.eigenvectors_right()   # compute right eigenvectors for a given lag time
                                for l in range(motion):
                                        its[l].append(time[l])
                                eigenvector.append(ev)
                        np.save('states_%d/frames_%d/its.npy'%(nstates,int(200000./stride)),its)
                        np.save('states_%d/frames_%d/eigenvector.npy'%(nstates,int(200000./stride)),eigenvector)
                        import matplotlib
                        matplotlib.use('Agg')
                        from matplotlib import pyplot as plt
                        plt.figure()
                        for m in range(motion):
                                plt.plot(interval*lagtimes,[n*interval for n in its[m]])
                        plt.xlabel('lagtime (ns)')
                        plt.ylabel('implied timescale (ns)')
                        plt.yscale('log')
                        plt.savefig('states_%d/frames_%d/implied_timescales.pdf'%(nstates,int(200000./stride)))
                        plt.close()
